# Netflix next banger

## Background

Netflix is a member-based internet entertainment service founded in 1997. The service allows members to watch tv-series, movies and documentaries through a streaming software on internet-connected devices against a monthly fee. Netflix’s content relies on a complex structure of algorithms that watches customer’s online use and are supposed to optimize satisfying decisions. Thus the company is in great need of user data. (Clares-Gavilán, Fernández-Manzano och Neira 2016, 568-576)

The company’s operational environment is characterized by hands-off approach to management. The workforce is heterogeneous, about 3,500 people who 
leaders trust to have the company’s best interests at heart and act correspondingly. (Gulati 2018, 4-13).  Furthermore a majority of Netflix’s content is from outside sources and the company’s operational flexibility is limited by outside partners (Ciejka 2018, 3-38).


## Problem description

As an effect of high competition and slow growth in the number of new subscribers Netflix share price has declined in fall 2019, hence Netflix wants to maximize the success of their next creation. The problem lies in not knowing what will be best received by the public. (Ny Teknik, 2019)

Additionally, by having some of its content come from outside sources, part of the problem becomes choosing the right content. If Netflix have to choose between several newly released movies or series, how will they know which one has the best chance of success?

In our project we will gather data from movies and their scripts. We will look for commonly used words, actors, genres and directors to predict which attributes Netflix should be the most interested in when creating their next movie and also which outside-produced movies they should add to their assortment. Finally upcoming movies will also be discussed with our analysed data in mind.


## Previous Studies
There are many different researches that has been carried out in the field using different approaches. Most of the studies found focuses on extracting data from social media to map how users online discussions, like reviews etc can predict a movies success. There are fewer studies that focuses on the attributes of a movie to explore how this is related to a movies success. 

In Ahmad et. al’s study Movie Success Prediction Using Data Mining (2017) the authors presents a study which aims to predict a movies success based on the weighted attributes budget, actors, director, producer, set locations, story writer, movie release day, competing movie releases at the same time, music, release location and target audience. The weighing is done using a mathematical model. It’s stated that “success cannot be predicted based on a particular attribute” and therefore weighing of multiple attributes will be integrated in this study as well. Furthermore the study gives a basic framework in how to value different attributes and what method to consider for the data mining. Lastly, the article shows how classification and clustering can be used in the context of prediction. It also gives a reference point to compare the results of this study to. 

Another similar study is Hammad Afzals article Prediction of Movies popularity Using Machine Learning Techniques (2016) which describes how machine learning approaches can predict an movies success based on datasets of IMDB scores and attributes. The most important finding of this article regarding the work of this report is that the attributes which contributed with most information was metascore and number of votes for each movie, Oscar awards won by the movies and the number of screens the movie is going to be screened which is considered in the choosing of attributes for this report. 

Furthermore Meenakshi et al also try to determine a movies success using several of the same attributes in their 2018 study. In the paper A Data mining Technique for Analyzing and Predicting the success of Movie they employ a decision tree together with clusters to categorize movies as either “Flop”, “Average” or “Hit”. They found that budget was no indication of how well a movie would be rated, and that genre was the most important node in their decision tree. (Meenakshi et al. 2018, 1-6)

The paper Predicting movie success and academy awards through sentiment and social network analysis talks about how much influence online communities have on the actual success of movies. The paper focuses on a model that that predicts academy award nominees by researching the opinions of online communities. Afterwards it uses the same approach to see if there is a correlation between online communities and movie success at the box office. By doing this, the study were able to predict different real events, such as nine academy award winners, by considering and using mentioned models on social networks in movie communities online.(Jonas Sebastian Krauss et al. 2008, 9-10). 


## Data collection

importing data from 4 separate tables: 
 - movies_metadata, containing several diffrerent attributes
 - title_principals, containing a connection between a movie and its cast members
 - movie_names, containing information about cast members
 - box_office // TODO 

In [1]:
import numpy as np
import csv

movie_data = []
title_principals = []
movie_names = []

with open('movies_metadata.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        movie_data.append(row) 

In [2]:
with open('title_principals.tsv', encoding="utf8") as tsvfile4:
    reader_titles = csv.reader(tsvfile4, delimiter='\t')
    for row in reader_titles:
        title_principals.append(row)

In [3]:
with open('movie_names.tsv', encoding="utf8") as tsvfile4:
    reader_rating = csv.reader(tsvfile4, delimiter='\t')
    for row in reader_rating:
        movie_names.append(row)
        
movie_data_np = np.array(movie_data[1:])

We begin our extraction by taking the movie_metadata table and selecting a few wanted attributes. The genres are clumped together in an ugly anonymous type-like object, so we split it up and save only the attribute names in an array. More filtering is done by first sorting the movies on imdb's id for later, and then removing any movie with a rating below 7.0. 

In [10]:
movie_data_selected_attributes = []

# Selecting attributes
for movie in movie_data_np:
    if(len(movie) > 22):
        movie_data_selected_attributes.append([movie[6], movie[3], movie[8], movie[14], movie[22], movie[23]])

print("Filter attributes done!")
        
movie_data_stripped_genres = []

# Rearranging genres
for movie in movie_data_selected_attributes:
    genre_arr = movie[1].split('\'')
    genres = []
    for i in range(5, len(genre_arr), 6):
        genres.append(genre_arr[i])       
    movie_data_stripped_genres.append([movie[0], genres, movie[2], movie[3], movie[4], movie[5]])

print("Genre rearranged done!")    

movies_above_7 = []

def custom_sort(t):
    return t[0]

def try_parse_float(input):
    try:
        number = float(input)
    except:
        number = -1
    return number

movie_data_stripped_genres.sort(key=custom_sort)

# Filter out movies with rating < 7.0
for movie in movie_data_stripped_genres:
    if(try_parse_float(movie[4]) >= 7.0):
        movies_above_7.append(movie)

print("Filter movies below 7 done!")
movies_above_7 = movies_above_7[3:]

Filter attributes done!
Genre rearranged done!
Filter movies below 7 done!


We define a function to help compare on ids. All imdb-ids consist of "xx0124002" where x is a letter. With this function we iterate through title_principals, the list with all connections, and movies_above_7, our filtered, sorted, list of movies. The connection is made by comparing ids from both lists, where movie_id iterates to "catch up" to title_id since many there are many more titles than movies in our lists.

The list with movie list is then filtered to remove any movie that did not have a corresponding crew.

In [12]:
def get_number(string):
    try:
        return (int) (string[2:])
    except:
        return 0
    
movies_with_crew = []

# Iterate through title_principals and movies_above_7, join on id.
j = 0
movie_id = 0
for i in range(len(title_principals)):
    title_id = get_number(title_principals[i][0])
    while(movie_id < title_id):
        j += 1
        if(j > len(movies_above_7) -1):
            break
        movie_id = get_number(movies_above_7[j][0])
    if(movie_id == title_id):
        if(j > len(movies_above_7) -1):
            break
        movies_above_7[j].append(title_principals[i][2])
        
movies_with_actors = []

# Remove non-matches, missing values
for movie in movies_above_7:
    if(len(movie) > 6):
        movies_with_actors.append(movie)

This list is saved as "movies_and_crew.csv" to not have to do this tedious task again...

In [15]:
with open('movies_and_crew.csv', 'w', encoding='utf-8') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(movies_with_actors)

Read movies_and_crew:

In [19]:
movies_and_crew = []

with open('movies_and_crew.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        if(len(row) > 0):
            movies_and_crew.append(row)


Filter out movies before 1960. Remove all non-actors from movie_names. Defines a function to look for an array of actor ids and return corresponding names. Loop breaks early if index at actor_id is reached, since the actor list is sorted we can assume no actor is beyond their own id as index. 

In [63]:
movies_after_1960 = []

# Remove movies before 1960
for movie in movies_and_crew:
    if(try_parse_float(movie[3].split('-')[0]) >= 1960):
        movies_after_1960.append(movie)
    
movie_names_actors = []

# Remove non-actors
for name in movie_names:
    if("actor" in name[4]):
        movie_names_actors.append(name)
    
# Takes a list of actor-ids and returns an array of names
def get_actor_names(actorList):
    actors = []
    for actor in actorList:
        actor_id = get_number(actor)
        count = 0
        for name in movie_names_actors:
            count += 1
            if(count > actor_id):
                break
            if(actor == name[0]):
                actors.append(name[1])
                break
    return actors

Connect all actors in the movie on id and replace with names of actors in initial array. To do this, first the movie_names list is filtered to remove any non-actor.

In [67]:
movies_with_actor_names = []

count = 1
# Replace actor ids with names
for movie in movies_after_1960:
    actors = get_actor_names(movie[6:])
    movies_with_actor_names.append([movie[:5], actors])
    count += 1
    if(count % 2 == 0):
        print("Progress...", round(count/4965 * 100, 2), "%")

Progress... 0.04 %
Progress... 0.08 %
Progress... 0.12 %
Progress... 0.16 %
Progress... 0.2 %
Progress... 0.24 %
Progress... 0.28 %
Progress... 0.32 %
Progress... 0.36 %
Progress... 0.4 %
Progress... 0.44 %
Progress... 0.48 %
Progress... 0.52 %
Progress... 0.56 %
Progress... 0.6 %
Progress... 0.64 %
Progress... 0.68 %
Progress... 0.73 %
Progress... 0.77 %
Progress... 0.81 %
Progress... 0.85 %
Progress... 0.89 %
Progress... 0.93 %
Progress... 0.97 %
Progress... 1.01 %
Progress... 1.05 %
Progress... 1.09 %
Progress... 1.13 %
Progress... 1.17 %
Progress... 1.21 %
Progress... 1.25 %
Progress... 1.29 %
Progress... 1.33 %
Progress... 1.37 %
Progress... 1.41 %
Progress... 1.45 %
Progress... 1.49 %
Progress... 1.53 %
Progress... 1.57 %
Progress... 1.61 %
Progress... 1.65 %
Progress... 1.69 %
Progress... 1.73 %
Progress... 1.77 %
Progress... 1.81 %
Progress... 1.85 %
Progress... 1.89 %
Progress... 1.93 %
Progress... 1.97 %
Progress... 2.01 %
Progress... 2.05 %
Progress... 2.09 %
Progress... 2.1

Progress... 17.16 %
Progress... 17.2 %
Progress... 17.24 %
Progress... 17.28 %
Progress... 17.32 %
Progress... 17.36 %
Progress... 17.4 %
Progress... 17.44 %
Progress... 17.48 %
Progress... 17.52 %
Progress... 17.56 %
Progress... 17.6 %
Progress... 17.64 %
Progress... 17.68 %
Progress... 17.72 %
Progress... 17.76 %
Progress... 17.8 %
Progress... 17.84 %
Progress... 17.89 %
Progress... 17.93 %
Progress... 17.97 %
Progress... 18.01 %
Progress... 18.05 %
Progress... 18.09 %
Progress... 18.13 %
Progress... 18.17 %
Progress... 18.21 %
Progress... 18.25 %
Progress... 18.29 %
Progress... 18.33 %
Progress... 18.37 %
Progress... 18.41 %
Progress... 18.45 %
Progress... 18.49 %
Progress... 18.53 %
Progress... 18.57 %
Progress... 18.61 %
Progress... 18.65 %
Progress... 18.69 %
Progress... 18.73 %
Progress... 18.77 %
Progress... 18.81 %
Progress... 18.85 %
Progress... 18.89 %
Progress... 18.93 %
Progress... 18.97 %
Progress... 19.01 %
Progress... 19.05 %
Progress... 19.09 %
Progress... 19.13 %
Prog

Progress... 33.76 %
Progress... 33.8 %
Progress... 33.84 %
Progress... 33.88 %
Progress... 33.92 %
Progress... 33.96 %
Progress... 34.0 %
Progress... 34.04 %
Progress... 34.08 %
Progress... 34.12 %
Progress... 34.16 %
Progress... 34.2 %
Progress... 34.24 %
Progress... 34.28 %
Progress... 34.32 %
Progress... 34.36 %
Progress... 34.4 %
Progress... 34.44 %
Progress... 34.48 %
Progress... 34.52 %
Progress... 34.56 %
Progress... 34.6 %
Progress... 34.64 %
Progress... 34.68 %
Progress... 34.72 %
Progress... 34.76 %
Progress... 34.8 %
Progress... 34.84 %
Progress... 34.88 %
Progress... 34.92 %
Progress... 34.96 %
Progress... 35.01 %
Progress... 35.05 %
Progress... 35.09 %
Progress... 35.13 %
Progress... 35.17 %
Progress... 35.21 %
Progress... 35.25 %
Progress... 35.29 %
Progress... 35.33 %
Progress... 35.37 %
Progress... 35.41 %
Progress... 35.45 %
Progress... 35.49 %
Progress... 35.53 %
Progress... 35.57 %
Progress... 35.61 %
Progress... 35.65 %
Progress... 35.69 %
Progress... 35.73 %
Progre

Progress... 50.35 %
Progress... 50.39 %
Progress... 50.43 %
Progress... 50.47 %
Progress... 50.51 %
Progress... 50.55 %
Progress... 50.59 %
Progress... 50.63 %
Progress... 50.67 %
Progress... 50.72 %
Progress... 50.76 %
Progress... 50.8 %
Progress... 50.84 %
Progress... 50.88 %
Progress... 50.92 %
Progress... 50.96 %
Progress... 51.0 %
Progress... 51.04 %
Progress... 51.08 %
Progress... 51.12 %
Progress... 51.16 %
Progress... 51.2 %
Progress... 51.24 %
Progress... 51.28 %
Progress... 51.32 %
Progress... 51.36 %
Progress... 51.4 %
Progress... 51.44 %
Progress... 51.48 %
Progress... 51.52 %
Progress... 51.56 %
Progress... 51.6 %
Progress... 51.64 %
Progress... 51.68 %
Progress... 51.72 %
Progress... 51.76 %
Progress... 51.8 %
Progress... 51.84 %
Progress... 51.88 %
Progress... 51.92 %
Progress... 51.96 %
Progress... 52.0 %
Progress... 52.04 %
Progress... 52.08 %
Progress... 52.12 %
Progress... 52.17 %
Progress... 52.21 %
Progress... 52.25 %
Progress... 52.29 %
Progress... 52.33 %
Progres

Progress... 66.95 %
Progress... 66.99 %
Progress... 67.03 %
Progress... 67.07 %
Progress... 67.11 %
Progress... 67.15 %
Progress... 67.19 %
Progress... 67.23 %
Progress... 67.27 %
Progress... 67.31 %
Progress... 67.35 %
Progress... 67.39 %
Progress... 67.43 %
Progress... 67.47 %
Progress... 67.51 %
Progress... 67.55 %
Progress... 67.59 %
Progress... 67.63 %
Progress... 67.67 %
Progress... 67.71 %
Progress... 67.75 %
Progress... 67.79 %
Progress... 67.83 %
Progress... 67.88 %
Progress... 67.92 %
Progress... 67.96 %
Progress... 68.0 %
Progress... 68.04 %
Progress... 68.08 %
Progress... 68.12 %
Progress... 68.16 %
Progress... 68.2 %
Progress... 68.24 %
Progress... 68.28 %
Progress... 68.32 %
Progress... 68.36 %
Progress... 68.4 %
Progress... 68.44 %
Progress... 68.48 %
Progress... 68.52 %
Progress... 68.56 %
Progress... 68.6 %
Progress... 68.64 %
Progress... 68.68 %
Progress... 68.72 %
Progress... 68.76 %
Progress... 68.8 %
Progress... 68.84 %
Progress... 68.88 %
Progress... 68.92 %
Progr

Progress... 83.54 %
Progress... 83.59 %
Progress... 83.63 %
Progress... 83.67 %
Progress... 83.71 %
Progress... 83.75 %
Progress... 83.79 %
Progress... 83.83 %
Progress... 83.87 %
Progress... 83.91 %
Progress... 83.95 %
Progress... 83.99 %
Progress... 84.03 %
Progress... 84.07 %
Progress... 84.11 %
Progress... 84.15 %
Progress... 84.19 %
Progress... 84.23 %
Progress... 84.27 %
Progress... 84.31 %
Progress... 84.35 %
Progress... 84.39 %
Progress... 84.43 %
Progress... 84.47 %
Progress... 84.51 %
Progress... 84.55 %
Progress... 84.59 %
Progress... 84.63 %
Progress... 84.67 %
Progress... 84.71 %
Progress... 84.75 %
Progress... 84.79 %
Progress... 84.83 %
Progress... 84.87 %
Progress... 84.91 %
Progress... 84.95 %
Progress... 84.99 %
Progress... 85.04 %
Progress... 85.08 %
Progress... 85.12 %
Progress... 85.16 %
Progress... 85.2 %
Progress... 85.24 %
Progress... 85.28 %
Progress... 85.32 %
Progress... 85.36 %
Progress... 85.4 %
Progress... 85.44 %
Progress... 85.48 %
Progress... 85.52 %
Pr

Aaaaand we save it.

In [68]:
with open('movies_and_crew_names.csv', 'w', encoding='utf-8') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(movies_with_actor_names)

## Data analysis

*Document you choice and motivation for selected data mining method(s) here. Choose a data mining method(s) to use in Python code to perform an analysis of your chosen dataset. Describe why you chose the method(s) and what interesting things you have found from the analysis.*

*Replace the contents of this cell with your own text.*

In [3]:
# Add your own code

## Evaluation of results

*Document an evaluation your analysis results and describe how potentially actionable they are.*

*Replace the contents of this cell with your own text.*

In [ ]:
# Add your own code

## Schedule and description of project plan

*Rough schedule for the project beyond the pilot study presented in 3-5. This does not have to be advanced, you can simply provide an estimate based upon reported schedules for similar projects in the literature.*

*Replace the contents of this cell with your own text.*

## Ethical aspects that need to be considered

*Are there ethical aspects that need to be considered? Are there legal implications (e.g., personal data / GDPR)? Are there implications if the case organization is a business, public authority, or nonprofit entity?*

*Replace the contents of this cell with your own text.*